In [1]:
!git clone https://github.com/Rope-player/pytorch_advanced.git

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 548 (delta 5), reused 162 (delta 0), pack-reused 374
Receiving objects: 100% (548/548), 50.13 MiB | 29.30 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd "pytorch_advanced"

/content/pytorch_advanced


In [3]:
%cd "3_semantic_segmentation"

/content/pytorch_advanced/3_semantic_segmentation


In [4]:
import os
import urllib.request
import zipfile
import tarfile

In [5]:
data_dir = "./data/"
if not os.path.exists(data_dir):
  os.mkdir(data_dir)

In [6]:
weights_dir = "./weights/"
if not os.path.exists(weights_dir):
  os.mkdir(weights_dir)

In [7]:
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar") 

if not os.path.exists(target_path):
  urllib.request.urlretrieve(url, target_path)
    
  tar = tarfile.TarFile(target_path)  
  tar.extractall(data_dir)  
  tar.close()

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# PSPNet 구성 및 구현

PSPNet은 네 개의 모듈로 구성됨
- Feature
- Pyramid Pooling
- Decoder
- AuxLoss

### Feature

Encoder 모듈러도 불림. 이 모듈은 입력 화상의 특징을 파악하는 것을 목적으로 함. 375 \* 375 크기의 화상을 입력받으면 60 \* 60 크기의 화상을 출력. 이때 2048개의 화상 특징을 파악한 채널을 준비해야 함.

### Pyramid Pooling

PSPNet의 독창성을 보여주는 모듈. 

픽셀의 라벨을 구하기위해 다양한 크기의 주변 정보를 요구함.

예를 들어 하나의 픽셀로는 소의 등인지 말의 등인지 알 수가 없지만, 픽셀의 주위를 점진적으로 확대한 특징량을 보면 소인지 말인지 확인 할 수 있음. 

즉 픽셀의 물체 라벨을 구하려면 그 주변 뿐만 아니라 더 넓은 화상 정보가 필요함.

크게 네 가지 크기의 특징맵을 준비함. 
1. 화상 전체
2. 화상 절반
3. 화상의 1/3
4. 화상의 1/6

출력데이터 크기는 4096 \* 60 \* 60임.

### Decoder

업샘플링 모듈이라고도 함. Decoder의 목적은 두 가지임

1. Pyramid Pooling의 출력을 21 \* 60 \* 60 텐서로 변환
2. 위의 텐서를 원 입력 화상 크기에 맞도록 21 \* 475 \* 475로 변환

### AuxLoss

원래라면 위 세 단계로 시맨틱 부날은 이루어지지만, PSPNet 에서는 파라미터의 학습을 효율적으로 하기 위해 AuxLoss 모듈을 준비함.

이 모듈은 손실함수 계산을 보조함.

학습시에는 AuxLoss의 출력돠 Decoder의 출력 모두 화상의 정답 정보로 대응시켜 손실값을 계산. 이후 손실 값에 따른 오차 역전파법을 실시하여 네트워크의 결합 파라미터를 갱신.

AuxLoss는 Feature층의 중간까지 결과로 시맨틱 분할을 실시하여 분류 정확도는 떨어지나, 오차 역전파법 수행 시 Feature층의 중간까지의 네트워크 파라미터가 더 좋은 값이 되도록 도움.

## PSPNet 클래스 구현

PSPNet의 형태를 규정하는 파라미터 설정 인수로서 클래스만 취하고 나머지는 하드코딩할 것임. Feature 모듈은 다섯개의 서브 네트워크로 기타 모듈은 하나의 서브 네트워크로 구성.
- `feature_conv`
- `feature_res_1`
- `feature_res_2`
- `feature_dilated_res_1`
- `feature_dilated_res_2`

PSPNet의 클래스 메소드는 forward 뿐임.

In [9]:
class PSPNet(nn.Module):
  def __init__(self, n_classes):
    super(PSPNet, self).__init__()

    # 파라미터 설정
    block_config = [3, 4, 6, 3]  # resnet50
    img_size = 475
    img_size_8 = 60              # img_size의 1/8로 설정

    # 4개의 모듈을 구성하는 서브 네트워크 준비
    self.feature_conv = FeatureMap_convolution()
    self.feature_res_1 = ResidualBlockPSP(n_blocks=block_config[0], in_channels=128, mid_channels=64, out_channels=256, stride=1, dilation=1)
    self.feature_res_2 = ResidualBlockPSP(n_blocks=block_config[1], in_channels=256, mid_channels=128, out_channels=512, stride=2, dilation=1)
    self.feature_dilated_res_1 = ResidualBlockPSP(n_blocks=block_config[2], in_channels=512, mid_channels=256, out_channels=1024, stride=1, dilation=2)
    self.feature_dilated_res_2 = ResidualBlockPSP(n_blocks=block_config[3], in_channels=1024, mid_channels=512, out_channels=2048, stride=1, dilation=4)

    self.pyramid_pooling = PyramidPooling(in_channels=2048, pool_sizes=[6, 3, 2, 1], height=img_size_8, width=img_size_8)
    self.decode_feature = DecodePSPFeature(height=img_size, width=img_size, n_classes=n_classes)
    self.aux = AuxiliaryPSPlayers(in_channels=1024, height=img_size, width=img_size, n_classes=n_classes)

  def forward(self, x):
    x = self.feature_conv(x)
    x = self.feature_res_1(x)
    x = self.feature_res_2(x)
    x = self.feature_dilated_res_1(x)

    output_aux = self.aux(x)  # Feature 모듈의 중간을 Aux 모듈로

    x = self.feature_dilated_res_2(x)

    x = self.pyramid_pooling(x)
    output = self.decode_feature(x)

    return (output, output_aux)

# Feature 모듈 설명 및 구현(ResNet)

## Feature 모듈의 서브 네트워크 구성

Feature 모듈은 다섯 개의 서브 네트워크인 `FeatureMap_convolution`, 두 개의 `RasidualBlockPSP`, 두 개의 `dilated 판 RasidualBlockPSP`로 구성.

네 번째 서브 네트워크인 `dilated 판 RasidualBlockPSP`의 출력텐서 1024 \* 60 \* 60 이 AuxLoss 모듈로 출력되는 점에 주의. 

AuxLoss 모듈에서 이 출력 텐서로 픽셀별 클래스를 분류하고 그 손실 값을 Feature 모듈의 전반부 네 개의 서브 네트워크를 학습하는데 사용. 

### 서브 네트워크 FeatureMap_convolution

`FeatureMap_convolution`은 네 가지 요소로 구성됨

- 합성곱 층
- 배치 정규화
- ReLu를 세트로 하는 `conv2dBatchNormReLu`가 세개, 최대 풀링층

서브 네트워크 `FeatureMap_convolution`은 단순히 합성곱, 배치 정규화, 최대풀링으로 화상의 특징을 추출하는 역할을 함

### FeatureMap_convolution 구현

ReLu를 세트로 하는 `conv2dBatchNormReLu` 클래스 작성.

ReLu 구현시 `nn.ReLu(inplace = True)`의 inplace는 메모리 입출력을 보존하면서 계산할 지 설정하는 파라미터.

In [10]:
class conv2DBatchNormRelu(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
    super(conv2DBatchNormRelu, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=bias)
    self.batchnorm = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=True)
    # inplase 설정으로 입력을 저장하지 않고 출력을 계산하여 메모리 절약

  def forward(self, x):
    x = self.conv(x)
    x = self.batchnorm(x)
    outputs = self.relu(x)

    return outputs

`conv2DBatchNormRelu` 클래스를 사용하여 `FeatureMap_convolution` 클래스 작성.

In [11]:
class FeatureMap_convolution(nn.Module):
  def __init__(self):
    # 구성할 네트워크 준비
    super(FeatureMap_convolution, self).__init__()

    # 합성곱 층 1
    in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 3, 64, 3, 2, 1, 1, False
    self.cbnr_1 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

    # 합성곱 층 2
    in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 64, 3, 1, 1, 1, False
    self.cbnr_2 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

    # 합성곱 층 3
    in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 128, 3, 1, 1, 1, False
    self.cbnr_3 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size, stride, padding, dilation, bias)

    # MaxPooling
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

  def forward(self, x):
    x = self.cbnr_1(x)
    x = self.cbnr_2(x)
    x = self.cbnr_3(x)
    outputs = self.maxpool(x)
    
    return outputs

## ResidualBlockPSP

Feature 모듈을 구성하는 두 개의 `ResidualBlockPSP`, 그리고 두 개의 dilated `ResidualBlockPSP`를 구현.

`ResidualBlockPSP`는 `bottleNeckPSP` 클래스를 지나 `bottleNeckIdentifyPSP` 클래스를 여러번 반복하여 출력.

서브 네트워크 `ResidualBlockPSP` 을 구현.

In [12]:
class ResidualBlockPSP(nn.Sequential):
  def __init__(self, n_blocks, in_channels, mid_channels, out_channels, stride, dilation):
    super(ResidualBlockPSP, self).__init__()

    # bottleNeckPSP 준비
    self.add_module(
      "block1",
      bottleNeckPSP(in_channels, mid_channels, out_channels, stride, dilation)
    )

    # bottleNeckIdentifyPSP 반복 준비
    for i in range(n_blocks - 1):
      self.add_module(
        "block" + str(i+2),
        bottleNeckIdentifyPSP(out_channels, mid_channels, stride, dilation)
      )

## bottleNeckPSP 와 bottleNeckIdentifyPSP 

ResidualBlockPSP 클래스에서 사용되는 bottleNeckPSP 와 bottleNeckIdentifyPSP 구현.

두 클래스는 특징적인 네트워크 구조를 가짐. 두 클래스는 입력이 두 갈래로 나누어 지는데, 그 중 하나는 스킵 결합이라고 함. `bottleNeckPSP` 와 `bottleNeckIdentifyPSP`의 차이는 이 스킵 결합에 합성곱층이 들어가는지 여부임.

`bottleNeckPSP` 는 스킵결합에 합성곱층을 한 번 적용하지만,
`bottleNeckIdentifyPSP` 는 적용하지 않음

딥러닝에서, 심층 네트워크의 학습할 파라미터가 많으면 적을 때 보다 오차율이 적을 것이라고 생각 하지만 실제로는 그렇지 않은 경우가 있음. 

이것을 **열화 문제**라고 함.

열화 문제를 피하기 위해 ResidualBlock의 입력 x를 그대로 출력하는 스킵 결합을 준비하는 것임.

전체 네트워크의 오차를 줄일 수 있도록 아이디어를 고안해 낸 것이 스킵 결합을 가진 ResidualBlock임.

`ResidualBlockPSP` 에서는 합성곱 층에서 `dilation` 파라미터를 설정.
네 개의 `ResidualBlockPSP` 중에서 앞의 두개는 dilation을 1로, 뒤의 두 개는 2와 4로 설정 되있음.

일반적인 합성곱층은 dilation 이 1임. 합성곱층에 1이 아닌 다른 dilation 값을 넣은 것을 `dilated Convolution`이라고 부름. dilation 값이 큰 `dilated Convolution` 일수록 더 거시적인 특징을 추출해 냄.

In [13]:
class conv2DBatchNorm(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
    super(conv2DBatchNorm, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=bias)
    self.batchnorm = nn.BatchNorm2d(out_channels)

  def forward(self, x):
    x = self.conv(x)
    outputs = self.batchnorm(x)

    return outputs


class bottleNeckPSP(nn.Module):
  def __init__(self, in_channels, mid_channels, out_channels, stride, dilation):
      super(bottleNeckPSP, self).__init__()

      self.cbr_1 = conv2DBatchNormRelu(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
      self.cbr_2 = conv2DBatchNormRelu(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=dilation, dilation=dilation, bias=False)
      self.cb_3 = conv2DBatchNorm(mid_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

      # 스킵 결합
      self.cb_residual = conv2DBatchNorm(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, dilation=1, bias=False)

      self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
      conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
      residual = self.cb_residual(x)

      return self.relu(conv + residual)


class bottleNeckIdentifyPSP(nn.Module):
  def __init__(self, in_channels, mid_channels, stride, dilation):
      super(bottleNeckIdentifyPSP, self).__init__()

      self.cbr_1 = conv2DBatchNormRelu(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
      self.cbr_2 = conv2DBatchNormRelu(mid_channels, mid_channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation, bias=False)
      self.cb_3 = conv2DBatchNorm(mid_channels, in_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
      self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
      conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
      residual = x
      
      return self.relu(conv + residual)

# Pyramid Pooling 모듈 설명 및 구현

### Pyramid Pooling 모듈의 서브 네트워크 구조

이곳에서는 Feature 모듈에서 출력된 텐서를 다섯개로 분기시킴. 가장 위의 분기는 Adaptive Average Pooling층으로 보내짐. 이곳에서는 화상을 지정된 크기로 Average Pooling을 함.

다섯 분기중 네 개는 출력이 각각 6, 3, 2, 1인 Adaptive Average Pooling 층으로 보내짐. (6 \* 6, 3 \* 3, 2 \* 2, 1 \* 1)

이렇게 점점 커지는 모양을 보고 피라미드와 비슷하여 붙여진 이름임.

Average Pooling 층을 통과한 텐서는 conv2DBatchNormReLu 클래스를 지나 업샘플층에 도달하고 작아진 특징맵의 크기를 60 \* 60 으로 확대함.

다섯 분기중 마지막 한 개는 입력을 그대로 출력하여 위 네개의 분기와 결합시킴.

### PyramidPooling 클래스 구현

Pyramid Pooling 모듈은 `PyramidPooling` 클래스 만으로 구축함.

In [14]:
class PyramidPooling(nn.Module):
  def __init__(self, in_channels, pool_sizes, height, width):
    super(PyramidPooling, self).__init__()

    # forward에서 사용하는 화상 크기
    self.height = height
    self.width = width

    # 각 합성곱층의 출력 채널 수
    out_channels = int(in_channels / len(pool_sizes))

    # 합성곱층 작성
    # 원래는 for 문으로 구성하는게 좋음.
    # pool_sizes: [6, 3, 2, 1]
    self.avpool_1 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[0])
    self.cbr_1 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

    self.avpool_2 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[1])
    self.cbr_2 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

    self.avpool_3 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[2])
    self.cbr_3 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

    self.avpool_4 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[3])
    self.cbr_4 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)

  def forward(self, x):

    out1 = self.cbr_1(self.avpool_1(x))
    out1 = F.interpolate(out1, size=(self.height, self.width), mode="bilinear", align_corners=True)

    out2 = self.cbr_2(self.avpool_2(x))
    out2 = F.interpolate(out2, size=(self.height, self.width), mode="bilinear", align_corners=True)

    out3 = self.cbr_3(self.avpool_3(x))
    out3 = F.interpolate(out3, size=(self.height, self.width), mode="bilinear", align_corners=True)

    out4 = self.cbr_4(self.avpool_4(x))
    out4 = F.interpolate(out4, size=(self.height, self.width), mode="bilinear", align_corners=True)

    # 최종 결합시킬 dim = 1 으로 채널 수 차원에서 결합
    output = torch.cat([x, out1, out2, out3, out4], dim=1)

    return output

# Decoder, AuxLoss 모듈 설명 및 구현

PSPNet의 Decoder 및 AuxLoss 모듈을 구성하는 서브 네트워크 구조 구현.

## Decoder 및 AuxLoss 모듈 구조

Decoder 및 AuxLoss 모듈은 이전에 출력된 텐서 정보를 Decode 합. 텐서 정보를 읽은 후 픽셀별로 물체 라벨을 클래스 분류로 추정하고 마지막으로 화상 크기를  원래의 475 * 475로 업샘플링 함.

Decoder 와 AuxLoss 모듈 모두 동일한 네트워크 구성임. 

conv2DBatchNormReLu 클래스를 통과한 후 드롭아웃 층을 지나 합성곱층을 통과하여 텐서 크기가 21 \* 60 \* 60 이 됨. 마지막으로 업샘플층을 지나 원래의 475 사이즈의 화상으로 확대.

## Decoder 및 AuxLoss 모듈 구현

In [15]:
class DecodePSPFeature(nn.Module):
  def __init__(self, height, width, n_classes):
    super(DecodePSPFeature, self).__init__()

    # forward에 사용하는 화상 크기
    self.height = height
    self.width = width

    self.cbr = conv2DBatchNormRelu(in_channels=4096, out_channels=512, kernel_size=3, stride=1, padding=1, dilation=1, bias=False)
    self.dropout = nn.Dropout2d(p=0.1)
    self.classification = nn.Conv2d(in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

  def forward(self, x):
    x = self.cbr(x)
    x = self.dropout(x)
    x = self.classification(x)
    output = F.interpolate(x, size=(self.height, self.width), mode="bilinear", align_corners=True)

    return output


class AuxiliaryPSPlayers(nn.Module):
  def __init__(self, in_channels, height, width, n_classes):
    super(AuxiliaryPSPlayers, self).__init__()

    # forward에 사용하는 화상 크기
    self.height = height
    self.width = width

    self.cbr = conv2DBatchNormRelu(in_channels=in_channels, out_channels=256, kernel_size=3, stride=1, padding=1, dilation=1, bias=False)
    self.dropout = nn.Dropout2d(p=0.1)
    self.classification = nn.Conv2d(in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

  def forward(self, x):
    x = self.cbr(x)
    x = self.dropout(x)
    x = self.classification(x)
    output = F.interpolate(x, size=(self.height, self.width), mode="bilinear", align_corners=True)

    return output

마지막 클래스 분류시 전결합층을 사용하지 않고 클래스 수와 동일하게 21을 출력 채널로 하는 커널크기 1의 합성곱층을 사용하는 것이 특징임.

마지막으로 네트워크 모델인 PSPNet 클래스의 인스턴스를 작성하고 오류 없이 계산되는지 확인.

In [16]:
# 모델 정의
net = PSPNet(n_classes=21)
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec

In [17]:
# 더미 데이터
batch_size = 2
dummy_img = torch.rand(batch_size, 3, 475, 475)

# 계산
outputs = net(dummy_img)
print(outputs)

(tensor([[[[ 6.2830e-01,  6.0226e-01,  5.7622e-01,  ...,  5.0809e-01,
            4.7802e-01,  4.4794e-01],
          [ 6.4608e-01,  6.1405e-01,  5.8203e-01,  ...,  5.1379e-01,
            4.8246e-01,  4.5114e-01],
          [ 6.6386e-01,  6.2585e-01,  5.8784e-01,  ...,  5.1948e-01,
            4.8690e-01,  4.5433e-01],
          ...,
          [ 4.3632e-01,  4.0551e-01,  3.7469e-01,  ...,  6.0906e-01,
            6.1720e-01,  6.2535e-01],
          [ 4.7366e-01,  4.3830e-01,  4.0295e-01,  ...,  6.1364e-01,
            6.1677e-01,  6.1989e-01],
          [ 5.1099e-01,  4.7110e-01,  4.3121e-01,  ...,  6.1822e-01,
            6.1633e-01,  6.1444e-01]],

         [[ 4.5468e-01,  4.4113e-01,  4.2758e-01,  ...,  6.0992e-01,
            6.6600e-01,  7.2208e-01],
          [ 4.2980e-01,  4.1977e-01,  4.0975e-01,  ...,  6.1643e-01,
            6.6121e-01,  7.0599e-01],
          [ 4.0491e-01,  3.9841e-01,  3.9191e-01,  ...,  6.2294e-01,
            6.5642e-01,  6.8991e-01],
          ...,
    